In [1]:
import streamlit as st
import os
import time
from crewai import Agent, Task, Crew, Process
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
from crewai_tools import tool

from docx import Document
from io import BytesIO
import base64

import pandas as pd
from IPython.display import Markdown, display

from dotenv import load_dotenv
#load_dotenv("./.env")
load_dotenv("/home/jovyan/.env")

os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")
os.environ["SLIP_AUTH_URL"] = os.environ.get("SLIP_AUTH_URL")
os.environ["SLIP_USERNAME"] =os.environ.get("SLIP_USERNAME")
os.environ["SLIP_PASSWORD"] =  os.environ.get("SLIP_PASSWORD")
os.environ["SLIP_CLIENT_ID"]= os.environ.get("SLIP_CLIENT_ID")
os.environ["SLIP_APP_NAME"]=os.environ.get("SLIP_APP_NAME")
os.environ["SERPER_API_KEY"] = "25d43901c968b0f8830387b7ed72231673428b9c"

In [2]:
from llm_utils import get_llm
llm = get_llm('gpt-4o-0513')

In [3]:
df=pd.read_csv('./data1.csv')
df

,mcid,dos,codeIndex,code,codeDesc,type,value
0,408478586,2017-07-17,8340,I10,Essential (primary) hypertension,diag,NaN
1,408478586,2017-07-17,14172,M545,Low back pain,diag,NaN
2,408478586,2017-11-29,11042,L309,"Dermatitis, unspecified",diag,NaN
3,408478586,2018-02-05,13402,M25512,Pain in left shoulder,diag,NaN
4,408478586,2018-02-05,15004,M75102,Unspecified rotator cuff tear or rupture of le...,diag,NaN
...,...,...,...,...,...,...,...
5313,432654279,2019-12-04,78782,4548-4_H,HbA1c MFr Bld,lab,8.800003
5314,408478586,2017-07-27,78785,4548-4_UNK,HbA1c MFr Bld,lab,5.600000
5315,408478586,2017-11-30,78785,4548-4_UNK,HbA1c MFr Bld,lab,5.400001
5316,408478586,2018-11-14,78785,4548-4_UNK,HbA1c MFr Bld,lab,5.500000


In [4]:
df1=df.sort_values(by=['mcid','codeIndex','dos'])

In [5]:
df1=df1[df1['mcid']==49958862]

In [6]:
df1[df1['type']=='lab']

,mcid,dos,codeIndex,code,codeDesc,type,value
5302,49958862,2019-02-15,78782,4548-4_H,HbA1c MFr Bld,lab,10.999997
5303,49958862,2019-05-15,78782,4548-4_H,HbA1c MFr Bld,lab,12.299999
5304,49958862,2019-08-14,78782,4548-4_H,HbA1c MFr Bld,lab,12.000002
5305,49958862,2019-11-15,78782,4548-4_H,HbA1c MFr Bld,lab,12.000002


In [7]:
df_json = df1.to_json()
import json


In [8]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()


In [9]:
@tool
def patientRecordSelect_tool(df_list, mcid):
    """Select the patient's medical records for patient with given mcid include all medical code types.
    'dos': date of service.
    'code': ICD10 diagnosis codes.
    'codeDesc': description of code.
    'type': type of code.
    'value': lab test results
    """
    df = pd.DataFrame(df_list)
    output = df[df['mcid'] == mcid][['dos', 'code', 'codeDesc','type','value']]
    return output

patientRecordSelect_agent = Agent(
    role="Get patient's medical records for all medical codes type",
    goal="For a given patient identified by mcid, get this patient's medical recordes "
         "from a given data using tool patientRecordSelect_tool.",
    backstory="You use given mcid to find the corresponding medical records.",
    tools=[patientRecordSelect_tool],
    allow_delegation=False,
    llm=llm  
)



summary_agent = Agent(
    role="Summarizer",
    goal="Summarize patient's medical records. Find out what diseases this patient had or having, and possible diseases may happen soon. ",
    tools = [scrape_tool, search_tool],
    verbose=True,
    llm = llm,
    backstory=(
        "As a medical doctor, you have a new patient who brings to you his/her past medical records."
        "Through analyzing this patient's medical records, you find what diseases this patient had in the past, "
        "what chronic desease this patient are having, "
        "and what desease this patient may have in the near future."
    )
)



commonDisease_agent = Agent(
    role="Detector of common diseases",
    goal="Find out if the given patient has diabetes and chronic kidney disease.",
    tools = [scrape_tool, search_tool],
    verbose=True,
    llm = llm,
    backstory=(
        "Diabetes, chronic kidney disease are very common, especially among older people. "
        "We are dealing with medicare population. Analyzing a patient's medical records to see if this patient has these diseases. "
        "If this patient has any of the mentioned diseases, please suggest treatment plan, one for each detected disease."
    )
)



In [10]:
patientRecordSelect_task = Task(
    description="Get the medical records for the given patient {mcid} from file {df_list}. These medical records will be used in later steps",
    expected_output="a table with columns: date of service, code, code description,type,lab result value",
    agent=patientRecordSelect_agent,
    callback=lambda output: save_agent_result("Patient Records", output)
)

summary_task = Task(
    description=(
        "1. Analyze the patient's medical records.\n"
        "2. Provide a summary of past, present, and possible future diseases based on the provided information."
        "3. Limit the length to 200 words."
    ),
    expected_output="A summary of patient's medical conditions.",
    agent=summary_agent,
    context=[patientRecordSelect_task],
    callback=lambda output: save_agent_result("Summary", output)
)

commonDisease_task = Task(
    description=(
        "1. Analyze the patient's medical records.\n"
        "2. Answer yes or no for diabetes and chronic kidney disease."
        "3. If the answer is yes, suggest treatment plan and dietary restrictions. When searching the internet, make sure check out Mayo Clinic website."
        "4. Limit the length to 200 words."
    ),
    expected_output="Yes or no answer to common chronic diseases. Suggestions if the answer is yes.",
    agent=commonDisease_agent,
    context=[patientRecordSelect_task, summary_task],
    callback=lambda output: save_agent_result("Common Diseases", output)
)

In [11]:
chatbot_agent = Agent(
    role="Medical Information Chatbot",
    goal="Provide accurate and helpful responses to user questions about the patient's medical condition or general medical queries",
    tools=[search_tool, scrape_tool],
    verbose=True,
    llm=llm,
    backstory=(
        "You are an AI-powered medical chatbot with the ability to analyze and interpret provided medical records, "
        "summaries, and assessments. You can answer questions "
        "specific to a patient as well as general medical queries. You have access to the outputs of "
        "previous analyses including patient record selection, medical summary, and common disease assessment. "
        "You can also search for additional information when needed to provide comprehensive answers."
    )
)

In [12]:
chatbot_task = Task(
    description=(
        "As a medical chatbot, your task is to answer '{user_question}' asked by the user accurately and helpfully. Follow these steps:\n"
        "1. Review the following patient-specific information:\n"
        "   a. Patient record selection results\n"
        "   b. Medical summary\n"
        "   c. Common disease assessment\n"
        "2. For the user question:\n"
        "   a. First, attempt to answer using the patient-specific information provided.\n"
        "   b. If the patient-specific information is insufficient, then use your general medical knowledge or perform a web search.\n"
        "   c. Provide a clear, concise, and accurate answer, prioritizing patient-specific information when available.\n"
        "   d. Ensure your response is easy for a non-medical professional to understand.\n"
        "   e. If the question is outside your scope or requires immediate medical attention, advise the user to consult a healthcare professional.\n"
        "   f. If you need to make any assumptions or if there's any ambiguity, clearly state this in your response."
    ),
    expected_output="A clear, informative, and accurate response to the user's question, prioritizing patient-specific information when available.",
    agent=chatbot_agent,
    output_file="response.txt",
    context=[patientRecordSelect_task, summary_task, commonDisease_task]
)

In [13]:
analysis_crew = Crew(
    agents=[patientRecordSelect_agent, summary_agent, commonDisease_agent],
    tasks=[patientRecordSelect_task, summary_task, commonDisease_task],
    verbose=1
)

# Create separate Crew for chatbot
chatbot_crew = Crew(
    agents=[chatbot_agent],
    tasks=[chatbot_task],
    verbose=1
)

2024-08-05 13:02:08,196 - 140097490958144 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [14]:
import json

results_store = {}

def save_agent_result(agent_name, result):
    results_store[agent_name] = result

def get_agent_result(agent_name):
    return results_store.get(agent_name, "No result available")

In [15]:
import gradio as gr

In [16]:
def process_mcid(mcid_input):
    global mcid
    mcid = mcid_input
    df_list = json.loads(df_json)
    
    analysis_crew.kickoff({"df_list": df_list, "mcid": mcid})
    
    return (get_agent_result("Patient Records"),
            get_agent_result("Summary"),
            get_agent_result("Common Diseases"))

def chat_with_bot(user_question):
    if not results_store:
        return "Please enter a patient MCID first."
    
    chatbot_response = chatbot_crew.kickoff({"user_question": user_question})
    return chatbot_response

with gr.Blocks() as demo:
    gr.Markdown("# Medical Record Analysis and Chatbot")
    
    with gr.Tab("Patient Analysis"):
        mcid_input = gr.Textbox(label="Enter Patient MCID")
        analyze_button = gr.Button("Analyze Patient Records")
        
        with gr.Tab("Patient Records"):
            records_output = gr.Textbox(label="Patient Records")
        with gr.Tab("Summary"):
            summary_output = gr.Textbox(label="Patient Summary")
        with gr.Tab("Common Diseases"):
            diseases_output = gr.Textbox(label="Common Diseases Assessment")
        
        analyze_button.click(process_mcid, 
                             inputs=mcid_input, 
                             outputs=[records_output, summary_output, diseases_output])
    
    with gr.Tab("Chat with Bot"):
        chat_input = gr.Textbox(label="Ask a question about the patient or any medical query")
        chat_button = gr.Button("Ask")
        chat_output = gr.Textbox(label="Chatbot Response")
        
        chat_button.click(chat_with_bot, inputs=chat_input, outputs=chat_output)



In [17]:
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://e90f56418d2bc1a393.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


 [2024-08-05 13:02:43][DEBUG]: == Working Agent: Get patient's medical records
 

I encountered an error while trying to use the tool. This was the error: patientRecordSelect_tool() missing 1 required positional argument: 'df_list'.
 Tool patientRecordSelect_tool accepts these inputs: patientRecordSelect_tool() - Select the patient's medical records for patient with given mcid.     'dos': date of service.     'code': ICD10 diagnosis codes.     'codeDesc': description of code.     'type': type of code.     'value': lab test results     

 

I encountered an error while trying to use the tool. This was the error: patientRecordSelect_tool() got an unexpected keyword argument '2697'.
 Tool patientRecordSelect_tool accepts these inputs: patientRecordSelect_tool() - Select the patient's medical records for patient with given mcid.     'dos': date of service.     'code': ICD10 diagnosis codes.     'codeDesc': description of code.     'type': type of code.     'value': lab test results     

 